In [51]:
import cv2
import numpy as np
import os
import glob
import math
from numpy import linalg as la

Parameters

In [113]:
# focal length
FOCAL_LENGTH = 668
# cylinder radius, it's much better to use focal length
CYLINDER_RADIUS = FOCAL_LENGTH
RANSAC_THRESHHOLD = 2.0


Get Data from folder

In [40]:
imgspath = glob.glob(os.path.join('img/gym/2-2','*.jpg'))
imgs = [cv2.imread(i) for i in imgspath]

In [41]:
img = imgs[0]
new_img = np.zeros(img.shape)
project_index = np.zeros(img.shape[0:2])
project_index.shape

(3264, 4912)

In [107]:
img = imgs[0]
img = cv2.resize(img, (img.shape[1]//10, img.shape[0]//10))
new_img = np.zeros(img.shape)


In [105]:
def project_to_cylinder(img, f = FOCAL_LENGTH, s = CYLINDER_RADIUS):
    height, width = img.shape[:2]
    cylinder_proj = np.zeros(shape=img.shape, dtype=np.uint8)
    
    for y in range(height):
        py = -y + height//2
        for x in range(width):
            px = x - width//2
            cylinder_proj[-int(s*py/math.sqrt(px**2+f**2) - height//2)][int(s*math.atan(px/f) + width//2)] = img[y][x]
    
    return cylinder_proj

In [142]:
def crop_image_black_edge(img, f = FOCAL_LENGTH, s = CYLINDER_RADIUS):
    h, w = img.shape[:2]
    top = -int(s*(h//2)/math.sqrt((-w//2)**2+f**2) - h//2)
    down = -int(s*(-h//2)/math.sqrt((-w//2)**2+f**2) - h//2)
    left = int(s*math.atan((-w//2)/f) + w//2)
    right = int(s*math.atan((w//2)/f) + w//2)
    crop_img = img[top:down, left:right] # crop 4 sides
    # crop_img = img[:, left:right] # only crop 2 sides, left to right

    return crop_img

In [116]:
# it's better to use random sample
def nonRANSAC(mp, th = 2): # mp = match pairs set of all images
    best_mp = []
    
    for i in range(len(mp)): # every pairs of different img img_x & img_x+1
        best_vote = 0
        best_mp.append(mp[i][0])
        for j in range(len(mp[i])): # find the best pair of the two images
            vote = 0
            temp_pair = np.array(mp[i][:, 0:2])
            trans_y = mp[i][j][2] - mp[i][j][0]
            trans_x = mp[i][j][3] - mp[i][j][1]
            temp_pair[0] += trans_y 
            temp_pair[1] += trans_x
            for k in range(len(temp_pair)):
                if(k == j):
                    continue
                if(la.norm(temp_pair[k]) < th):
                    vote += 1

            if(vote > best_vote):
                best_vote = vote
                best_mp[i] = mp[i][j]

    return best_mp
        

In [147]:
#  img2       img1            img2+img1
#  _____      _____           _____
# |     |    |.    |  __\    |    _|___   shift = (c,d) - (a,b)
# |    .|    |     |     \   |   |.|   |
# |_____|    |_____|  __ /   |___|_|   |
#  (c,d)      (a,b)     /        |_____|
# 
# y shift > 0, means img1 move down
# x shift > 0, means img1 move right
# in our case, our image are counterclockwise and equal size, so x shift must > 0

def stitch_image(img1, img2, shift): # img1 may be bigger, img2 is always the same size
    img1_padding = [
        (shift[0], 0) if shift[0] > 0 else (0, -shift[0]), 
        (shift[1], 0) if shift[1] > 0 else (0, -shift[1]), 
        (0, 0)
    ]
    padded_img1 = np.lib.pad(img1, img1_padding, 'constant', constant_values=0)

    # h2p = padded_img1.shape[0] - img2.shape[0]
    # w2p = padded_img1.shape[1] - img2.shape[1]
    # img2_padding = [
    #     (h2p, 0) if shift[0] < 0 else (0, -h2p), 
    #     (w2p, 0) if shift[1] < 0 else (0, -w2p), 
    #     (0, 0)
    # ]
    # padded_img2 = np.lib.pad(img2, img2_padding, 'constant', constant_values=0)


    padded_img1[:img2.shape[0], :img2.shape[1]] = img2

    return padded_img1

    # for i in range(len(match_points)): 
    #     img_concat = imgs[i+1]

    #     img_w = imgs[i].shape[1]
    #     for j in match_points[i]:
    #         img_concat = cv2.line(img_concat, (j[1], j[0]), (j[3]+img_w, j[2]), (255,0,0), 1)

    #     cv2.imwrite(f'result{i}_{i+1}.jpg', img_concat)

In [121]:
new_img = project_to_cylinder(img)

cv2.imwrite("my_projection.jpg", new_img)

True

In [145]:
crop_img = crop_image_black_edge(new_img)

crop_img.shape
cv2.imwrite("crop.jpg", crop_img)

True

In [146]:
shift = [50,70]
img1_padding = [
        (shift[0], 0) if shift[0] > 0 else (0, -shift[0]), 
        (shift[1], 0) if shift[1] > 0 else (0, -shift[1]), 
        (0, 0)
    ]
padded_img1 = np.lib.pad(crop_img, img1_padding, 'constant', constant_values=0)
padded_img1[:crop_img.shape[0], :crop_img.shape[1]] = crop_img
cv2.imwrite("padded_img1.jpg", padded_img1)

True

In [46]:


# result_imgs = []
# for _, img in enumerate(imgs):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     gray = np.float32(gray)
#     # 這個不能call
#     dst = cv2.cornerHarris(gray,2,3,0.1)
#     dst = cv2.dilate(dst,None)
#     img[dst>0.01*dst.max()] = [0,0,255]
#     result_imgs.append(img)

# for i, img in enumerate(result_imgs):
#     cv2.imwrite(os.path.join('result', f'img{i}.jpg'), img)
